# Search

> Search for topics using SearXNG - a popular open-source meta-search engine


In [ ]:
# | default_exp search

## Installing SearXNG

[Docker Installation Instructions](https://docs.searxng.org/admin/installation-docker.html#installation-docker)

## Running

SearXNG can be started thusly

```
docker run -it \
    -d -p 8080:8080 \
    -v "${PWD}/searxng:/etc/searxng" \
    -e "BASE_URL=http://localhost:8080/" \
    -e "INSTANCE_NAME=my-instance" \
    searxng/searxng
```


In [ ]:
# | export

import requests
import msgspec
import os

In [ ]:
# | hide

from rich.pretty import pprint

# Using SearXNG to get results back in json format

The easiest way to search a number of sources and get results back in JSON format is
to use SearXNG's open-source meta-search engine. JSON responses needs to be enabled in
SeacXNG (by editing `settings.yml` in order to work.)


In [ ]:
# Define the search query
query = "!news Continental"

# Define the SearXNG search API endpoint
api_endpoint = "http://localhost:8080/search"


# Define the request parameters
params = {
    "q": query,
    "format": "json",
    "language": "auto",
    "time_range": "month",
    "safesearch": 0,
}

search_results = []

for page in range(1, 3):
    params["pageno"] = page
    # Send the GET request to the SearXNG search API
    response = requests.get(api_endpoint, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the search results
        page_results = response.json()
        # Add search results to the end of the list
        search_results.extend(page_results["results"])
    else:
        print("Error occurred while performing the search.")

for i, result in enumerate(search_results):
    print(f"{i}: {result['title']}")

0: Continental Congress, families faced hardship during, after Revolutionary War stay in York
1: Slow sales drags Germany's Continental to a loss in Q1
2: Continental Backs Outlook Despite Net Loss
3: Continental AG Slips To Q1 Loss On Weak Sales; Sees Earnings Growth Ahead, Confirms FY24 View
4: Continental goes from profit to 53 million loss in the 1st quarter
5: Continental Confirms 1Q Preliminary Figures
6: Continental fined 100 million euros for role in VW emissions scandal
7: Why is the Olympic Torch going on a cross continental journey from Olympus to Paris?
8: Continental earnings: here's what to expect
9: “Sufficient potential for improvement” – Continental publishes Q1 2024 results
10: Spring Continental Market to return to grounds of Belfast City Hall this May
11: Auto parts supplier Continental agrees to pay €100m in diesel scandal
12: Continental earnings: here's what to expect
13: “Sufficient potential for improvement” – Continental publishes Q1 2024 results
14: Continent

### Format of data coming back


In [ ]:
pprint(search_results[0])

{
│   'url': 'https://www.bizjournals.com/dallas/news/2024/04/30/culpepper-cattle-co-continental-gin-building.html',
│   'title': "Culpepper Cattle Co. to open in Deep Ellum's historic Continental Gin Building",
│   'content': "Now they're expanding the concept for the first time ever, and leaders of UNCO feel like they've secured the perfect space ...",
│   'img_src': 'https://www.bing.com//th?id=OVFT.zH8tflpzozKSS8fwa6EDEi&pid=News&w=234&h=132&c=14&rs=2&qlt=90',
│   'metadata': '6 days ago | The Business Journals',
│   'engine': 'bing news',
│   'parsed_url': [
│   │   'https',
│   │   'www.bizjournals.com',
│   │   '/dallas/news/2024/04/30/culpepper-cattle-co-continental-gin-building.html',
│   │   '',
│   │   '',
│   │   ''
│   ],
│   'template': 'default.html',
│   'engines': ['bing news'],
│   'positions': [1],
│   'score': 1.0,
│   'category': 'news'
}

In [ ]:
# | export


class SearchResult(msgspec.Struct):
    title: str
    content: str
    url: str
    image_url: str
    source: str

    @classmethod
    def from_dict(cls, d):
        return cls(
            title=d["title"],
            content=d["content"],
            url=d["url"],
            image_url=d["img_src"],
            source=", ".join(d["engines"]),
        )


class SearchResults(msgspec.Struct):
    page: int
    results: list[SearchResult]

    @classmethod
    def from_list(cls, l: list[dict], page: int):
        return cls(
            page=page,
            results=[SearchResult.from_dict(d) for d in l],
        )

In [ ]:
# | export


def run_query(query: str, page: int, time_range: str = "month") -> list[SearchResult]:
    api_endpoint = os.environ["SEARCH_ENDPOINT"]
    params = {
        "q": query,
        "format": "json",
        "language": "auto",
        "time_range": time_range,
        "safesearch": 0,
        "pageno": page,
    }
    response = requests.get(api_endpoint, params=params)
    if response.status_code == 200:
        page_results = response.json()
        return [SearchResult.from_dict(d) for d in page_results["results"]]
    else:
        return []

In [ ]:
search_results = run_query("!news Trimble", 1, time_range="week")
pprint(search_results)

[
│   SearchResult(
│   │   title='Trimble (TRMB) Q1 Earnings Beat Estimates, Revenues Up Y/Y',
│   │   content='Trimble TRMB delivered first-quarter 2024 non-GAAP earnings of 64 cents per share, which surpassed the Zacks Consensus Estimate by 8.47% but decreased 11.1% on a year-over-year basis. Revenues of $953 ...',
│   │   url='https://finance.yahoo.com/news/trimble-trmb-q1-earnings-beat-174900575.html',
│   │   image_url=None,
│   │   source='bing news'
│   ),
│   SearchResult(
│   │   title='Trimble First Quarter 2024 Earnings: Revenues Beat Expectations, EPS Lags',
│   │   content='Revenue exceeded analyst estimates by 4.5%. Earnings per share (EPS) missed analyst estimates by 26%. Looking ahead, revenue ...',
│   │   url='https://finance.yahoo.com/news/trimble-first-quarter-2024-earnings-104006482.html',
│   │   image_url='https://www.bing.com//th?id=OVFT.X7XjenT6fhjjmbmZwGO6RS&pid=News&w=234&h=132&c=14&rs=2&qlt=90',
│   │   source='bing news'
│   ),
│   SearchResult(
│   │   title='Analysts Offer Insights on Technology Companies: Trimble (TRMB) and Robinhood Markets (HOOD)',
│   │   content='Analysts have been eager to weigh in on the Technology sector with new ratings on Trimble (TRMB – Research Report) and Robinhood Markets (HOOD – Research Report). Trimble (TRMB) In a report released ...',
│   │   url='https://markets.businessinsider.com/news/stocks/analysts-offer-insights-on-technology-companies-trimble-trmb-and-robinhood-markets-hood-1033334809',
│   │   image_url=None,
│   │   source='bing news'
│   ),
│   SearchResult(
│   │   title="ELD security a 'shared responsibility,' expert says, as logging devices ID'd as cyberattack vector",
│   │   content='Data theft is one of the biggest features of hacking into ELDs, but trucking and hacking capture the imagination in other ...',
│   │   url='https://www.ccjdigital.com/technology/article/15670056/eld-security-a-shared-responsibility',
│   │   image_url='https://www.bing.com//th?id=OVFT.f8qs1cviAwvuEeDYpcAwcC&pid=News&w=234&h=132&c=14&rs=2&qlt=90',
│   │   source='bing news'
│   ),
│   SearchResult(
│   │   title='Wall Street Analysts Are Bullish on Top Technology Picks',
│   │   content='There’s a lot to be optimistic about in the Technology sector as 3 analysts just weighed in on Trimble (TRMB – Research Report), Microstrategy (MSTR – Research Report) and Microsoft (MSFT – Research ...',
│   │   url='https://markets.businessinsider.com/news/stocks/wall-street-analysts-are-bullish-on-top-technology-picks-1033334124',
│   │   image_url=None,
│   │   source='bing news'
│   ),
│   SearchResult(
│   │   title='UNC Basketball Staff Loses Transfer Battle to John Calipari',
│   │   content='UNC basketball head coach Hubert Davis and his gang have missed out on yet another frontcourt transfer target in new Arkansas pledge Adou Thiero.',
│   │   url='https://www.si.com/college/unc/basketball-staff-loses-transfer-battle-john-calipari-5-24',
│   │   image_url=None,
│   │   source='bing news'
│   ),
│   SearchResult(
│   │   title="Asphalt Contractor's Top 30 Products 2024",
│   │   content='Every year there are a great number of new products that come to market for the asphalt and road building industry. They make ...',
│   │   url='https://www.forconstructionpros.com/asphalt/article/22892442/asphalt-contractors-top-30-products-2024',
│   │   image_url='https://www.bing.com//th?id=OVFT.CsNAYuGbWPNueHPbFGYiIS&pid=News&w=234&h=132&c=14&rs=2&qlt=90',
│   │   source='bing news'
│   ),
│   SearchResult(
│   │   title='oneNav announces the first L5-direct GNSS receiver technology',
│   │   content='Global navigation satellite system (GNSS) technology developer oneNav announces the launch of L5-direct™, a new GNSS product ...',
│   │   url='https://www.design-reuse.com/news/56182/onenav-l5-direct-gnss-receiver.html',
│   │   image_url='https://www.bing.com//th?id=OVFT.qhxUzUJOn-pi3i5qy1BYUC&pid=News&w=234&h=132&c=14&rs=2&qlt=90',
│   │   source='bing news'
│   ),
│   SearchResult(
│  

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()